In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('data.csv', names=["Timestamp", "Duration", "Type", "Level",
                                      "Client", "Client ID", "Query ID",
                                      "Query Name", "View", "Recursion",
                                      "Query Type", "Query", "Class",
                                      "Record Type", "Flags", "IP Address"],low_memory=False)
df.head()


,Timestamp,Duration,Type,Level,Client,Client ID,Query ID,Query Name,View,Recursion,Query Type,Query,Class,Record Type,Flags,IP Address
0,16-Apr-24,50:53.8,queries:,info:,client,@0x7fccd4084398,-4.013320e+18,(fmcdg2.db-schenker.biz):,view,recursive:,query:,fmcdg2.db-schenker.biz,IN,A,+,(103.58.120.120)
1,16-Apr-24,50:53.8,queries:,info:,client,@0x7fcd2c4568a8,-8.241670e+18,(ip-10-0-44-250.us-west-2.compute.regio-bayeri...,view,recursive:,query:,ip-10-0-44-250.us-west-2.compute.regio-bayeris...,IN,A,+,(103.58.120.120)
2,16-Apr-24,50:53.9,queries:,info:,client,@0x7fcce41535b8,-8.235850e+18,(amazon.com):,view,recursive:,query:,amazon.com,IN,A,+,(103.58.120.120)
3,16-Apr-24,50:53.9,queries:,info:,client,@0x7fcce415d148,-8.235850e+18,(wikipedia.org):,view,recursive:,query:,wikipedia.org,IN,A,+,(103.58.120.120)
4,16-Apr-24,50:53.9,queries:,info:,client,@0x7fcdd08263e8,-8.797800e+18,(e.bank.dbdialog.eu):,view,recursive:,query:,e.bank.dbdialog.eu,IN,A,+,(103.58.120.120)


In [33]:
# Get the shape of the DataFrame
data_shape = df.shape

# Display the shape
print(f"Number of rows: {data_shape[0]}")
print(f"Number of columns: {data_shape[1]}")

Number of rows: 3692
Number of columns: 16


In [34]:
null_values = df.isnull().sum()
print(null_values)

Timestamp      0
Duration       0
Type           0
Level          0
Client         0
Client ID      0
Query ID       0
Query Name     0
View           0
Recursion      0
Query Type     0
Query          0
Class          0
Record Type    0
Flags          0
IP Address     0
dtype: int64


In [35]:
# Display rows with any null values
rows_with_nulls = df[df.isnull().any(axis=1)]

# Display the rows with null values
print(rows_with_nulls)

Empty DataFrame
Columns: [Timestamp, Duration, Type, Level, Client, Client ID, Query ID, Query Name, View, Recursion, Query Type, Query, Class, Record Type, Flags, IP Address]
Index: []


In [36]:
# Drop rows with any null values
df.dropna(inplace=True)

# Verify that the rows with null values have been dropped
print(df.isnull().sum())
print(df.shape)

Timestamp      0
Duration       0
Type           0
Level          0
Client         0
Client ID      0
Query ID       0
Query Name     0
View           0
Recursion      0
Query Type     0
Query          0
Class          0
Record Type    0
Flags          0
IP Address     0
dtype: int64
(3692, 16)


In [37]:
# List of columns to select
columns_to_select = ["Timestamp", "Duration", "Client", "Client ID", "Query ID", "Query", "Class", "Record Type", "Flags", "IP Address"]

# Select the specified columns
selected_df = df[columns_to_select]

# Display the first few rows of the selected DataFrame
print(selected_df.head())

# Create a copy of the DataFrame
selected_columns = selected_df.copy()
selected_columns.shape

   Timestamp Duration  Client        Client ID      Query ID  \
0  16-Apr-24  50:53.8  client  @0x7fccd4084398 -4.013320e+18   
1  16-Apr-24  50:53.8  client  @0x7fcd2c4568a8 -8.241670e+18   
2  16-Apr-24  50:53.9  client  @0x7fcce41535b8 -8.235850e+18   
3  16-Apr-24  50:53.9  client  @0x7fcce415d148 -8.235850e+18   
4  16-Apr-24  50:53.9  client  @0x7fcdd08263e8 -8.797800e+18   

                                               Query Class Record Type Flags  \
0                             fmcdg2.db-schenker.biz    IN           A     +   
1  ip-10-0-44-250.us-west-2.compute.regio-bayeris...    IN           A     +   
2                                         amazon.com    IN           A     +   
3                                      wikipedia.org    IN           A     +   
4                                 e.bank.dbdialog.eu    IN           A     +   

         IP Address  
0  (103.58.120.120)  
1  (103.58.120.120)  
2  (103.58.120.120)  
3  (103.58.120.120)  
4  (103.58.120.120)  


(3692, 10)

In [38]:
import pandas as pd

# Define the function to format time
def format_time(time_str):
  try:
      # Split the time string into minutes, seconds, and tenths of a second
      minutes, rest = time_str.split(':')
      seconds, tenths = rest.split('.')

      # Convert to integers
      minutes = int(minutes)
      seconds = int(seconds)
      milliseconds = int(tenths) * 100  # Convert tenths of a second to milliseconds

      # Calculate total milliseconds
      total_milliseconds = (minutes * 60 * 1000) + (seconds * 1000) + milliseconds

      # Convert to timedelta
      time_delta = pd.to_timedelta(total_milliseconds, unit='ms')

      # Format the time
      formatted_time = f"{time_delta.components.hours:02}:{time_delta.components.minutes:02}:{time_delta.components.seconds:02}.{time_delta.components.milliseconds:03}"
      return formatted_time
  except Exception as e:
      print(f"Error formatting time: {time_str} - {e}")
      return None  # Return None for invalid entries

# Create a copy of the DataFrame
selected_columns = selected_df.copy()

# Apply the function to each row
selected_columns['time'] = selected_columns['Duration'].apply(format_time)

# Convert the 'time' column to timedelta type
selected_columns['time'] = pd.to_timedelta(selected_columns['time'])

# Format the 'time' column to remove "0 days"
selected_columns['time'] = selected_columns['time'].apply(lambda x: f"{x.components.hours:02}:{x.components.minutes:02}:{x.components.seconds:02}.{x.components.milliseconds:03}")

# Print the updated DataFrame
print(selected_columns)

      Timestamp Duration  Client        Client ID      Query ID  \
0     16-Apr-24  50:53.8  client  @0x7fccd4084398 -4.013320e+18   
1     16-Apr-24  50:53.8  client  @0x7fcd2c4568a8 -8.241670e+18   
2     16-Apr-24  50:53.9  client  @0x7fcce41535b8 -8.235850e+18   
3     16-Apr-24  50:53.9  client  @0x7fcce415d148 -8.235850e+18   
4     16-Apr-24  50:53.9  client  @0x7fcdd08263e8 -8.797800e+18   
...         ...      ...     ...              ...           ...   
3687  16-Apr-24  51:44.2  client  @0x7fccd44e5ef8 -9.015550e+18   
3688  16-Apr-24  51:44.2  client  @0x7fccf824c538 -8.235850e+18   
3689  16-Apr-24  51:44.2  client  @0x7fcd20352558  6.200840e+18   
3690  16-Apr-24  51:44.2  client  @0x7fcdecf88c88 -8.721890e+18   
3691  16-Apr-24  51:44.2  client  @0x7fccd409cd08 -2.905440e+18   

                                                  Query Class Record Type  \
0                                fmcdg2.db-schenker.biz    IN           A   
1     ip-10-0-44-250.us-west-2.compute.re

In [39]:
# Drop the 'Duration' column
selected_columns = selected_columns.drop('Duration', axis=1)

# Print the updated DataFrame
print(selected_columns)

      Timestamp  Client        Client ID      Query ID  \
0     16-Apr-24  client  @0x7fccd4084398 -4.013320e+18   
1     16-Apr-24  client  @0x7fcd2c4568a8 -8.241670e+18   
2     16-Apr-24  client  @0x7fcce41535b8 -8.235850e+18   
3     16-Apr-24  client  @0x7fcce415d148 -8.235850e+18   
4     16-Apr-24  client  @0x7fcdd08263e8 -8.797800e+18   
...         ...     ...              ...           ...   
3687  16-Apr-24  client  @0x7fccd44e5ef8 -9.015550e+18   
3688  16-Apr-24  client  @0x7fccf824c538 -8.235850e+18   
3689  16-Apr-24  client  @0x7fcd20352558  6.200840e+18   
3690  16-Apr-24  client  @0x7fcdecf88c88 -8.721890e+18   
3691  16-Apr-24  client  @0x7fccd409cd08 -2.905440e+18   

                                                  Query Class Record Type  \
0                                fmcdg2.db-schenker.biz    IN           A   
1     ip-10-0-44-250.us-west-2.compute.regio-bayeris...    IN           A   
2                                            amazon.com    IN           

In [40]:
import re
import pandas as pd

# Define the regular expressions
regex_patterns = {
  "service_specific_dns": r"^_ldap\._tcp\.dc\._msdcs\.[A-Z]+\.[A-Z]+$",
  "uuid_based_service_specific_dns": r"^_ldap\._tcp\.[0-9a-fA-F-]{36}\.domains\._msdcs\.[A-Z]+\.[A-Z]+$",
  "standard_hostname_domain": r"^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",
  "complex_subdomain_structure": r"^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",
  "standard_hostname_subdomain": r"^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",
  "simple_domain": r"^[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",  # Added to match simple domains like "amazon.com"
  "single_label_domain": r"^[a-zA-Z0-9-]+$",  # Added to match single label domains like "."
  "extended_subdomain_structure": r"^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",  # Extended subdomain structure
  "another_extended_subdomain_structure": r"^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",  # Another extended subdomain structure
  "ip_based_subdomain": r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",  # IP-based subdomain
  "complex_ip_based_subdomain": r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",  # Complex IP-based subdomain
  "subdomain_with_underscore": r"^[a-zA-Z0-9-_]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",  # Subdomain with underscore
  "complex_subdomain_with_underscore": r"^[a-zA-Z0-9-_]+\.[a-zA-Z0-9-_]+\.[a-zA-Z0-9-_]+\.[a-zA-Z0-9-_]+\.[a-zA-Z]{2,}$"  # Complex subdomain with underscore
}


# Extract query names
query_names = selected_columns['Query'].tolist()

# Function to match query names to regex patterns
def match_query_names(query_names, regex_patterns):
  matches = {}
  for name in query_names:
      for pattern_name, pattern in regex_patterns.items():
          if re.match(pattern, name):
              matches[name] = pattern_name
              break
  return matches

# Match the query names
matched_queries = match_query_names(query_names, regex_patterns)

# Print the results
for query, pattern_name in matched_queries.items():
  print(f"Query: {query} matches pattern: {pattern_name}")

Query: fmcdg2.db-schenker.biz matches pattern: standard_hostname_domain
Query: ip-10-0-44-250.us-west-2.compute.regio-bayerisch-schwaben.de matches pattern: another_extended_subdomain_structure
Query: amazon.com matches pattern: simple_domain
Query: wikipedia.org matches pattern: simple_domain
Query: e.bank.dbdialog.eu matches pattern: standard_hostname_subdomain
Query: moncompte.cogeco.ca matches pattern: standard_hostname_domain
Query: www.sparrow-jnu.zlgl.eoffice.gov.in matches pattern: extended_subdomain_structure
Query: lunaful.yorouto.com matches pattern: standard_hostname_domain
Query: sandbox-financing-v2.streckenkunde.net matches pattern: standard_hostname_domain
Query: rgftomongodbqasubversion.blob.core.windows.net matches pattern: another_extended_subdomain_structure
Query: tf-dev-pipeid-9b97a.datenbaeren-test.comp.transfracht.com matches pattern: another_extended_subdomain_structure
Query: hq-rdgateway.dbbahnbau.de matches pattern: standard_hostname_domain
Query: d19058ce9c

In [41]:
# Calculate the percentage of matched queries
total_queries = len(query_names)
matched_count = len(matched_queries)
percentage_matched = (matched_count / total_queries) * 100 if total_queries > 0 else 0

# Print the results
print(f"Total Queries: {total_queries}")
print(f"Matched Queries: {matched_count}")
print(f"Percentage of Queries Matched: {percentage_matched:.2f}%")

Total Queries: 3692
Matched Queries: 2742
Percentage of Queries Matched: 74.27%


In [42]:
# Get unique matched queries
matched_set = set(matched_queries.keys())

# Print the unique matched queries
print("Unique Queries that matched a pattern:")
for query in matched_set:
  print(query)

Unique Queries that matched a pattern:
demo.scoop.dbsystel.de
update.googleapis.com
shield.eu-central-1.myid.bahnpartner.eu
prdresources1.analyse-bahn.de
magentoeetpux.blob.core.windows.net
darkness_dc-telecom.s3.amazonaws.com
_ldap._tcp.dc._msdcs.BIRLASOFT.COM
safexis.app106.rp.roblox.com
ea2345fef3f3de46d9b5836e27dacd48-sb.portal-api.cleardive-nonprod.ntd.nintendo.com
newapplicant.db-schenker.de
edge-msgr-latest-shv-01-sof1.facebook.com
crawweb0430-c3s-jp2p-prod.147-92-163-102.search.line-apps.com
thepaperanggang_popularmovies.s3.amazonaws.com
mysitesv.xn--das-ist-grn-2hb.com
www.googleapis.com
sapcc.bahnliegenschaften.de
196722-kk.dbbahn.ru
iphonect.trassenplaner.de
ip-172-16-41-27.eu-west-2.compute.railport.fr
playdesign.nintendo.com
infwtctwprd.dbcargo.ag
ip-10-0-116-22.ec2.db-cargo.pl
twit.transfracht.com
phs1.ak-schnellbus.de
bb16-email.kucoin.com
httpcert-testing.wifionice.de
riad-luxe-56-marrakesh.booked003.s3.amazonaws.com
ip-172-16-191-104.eu-west-1.compute.db-cargo.pl
stag-

In [43]:
# Get unique unmatched queries
unmatched_queries = set(query_names) - matched_set

# Print the unique unmatched queries
print("Unique Queries that did not match any pattern:")
for query in unmatched_queries:
  print(query)

Unique Queries that did not match any pattern:
sentry.ext.usc1.stg.k8s.streckenkunde.net
epdg.epc.mnc002.mcc404.pub.3gppnetwork.org
edge.scratch-b2152.aws.dev.av.lyft.net
principal-mapping.us-east-1.gamma.kendra.aws.dbjomp.com
vreagles1868.sphere.online.judge.s3.amazonaws.com
message-broker.at-2246.veib.sab-test.comp.db-schenker.dk
configuration.cell-1.us-west-2.gamma.gardner.dbservicepoint.de
message-broker.at-2576.veib.sab-test.comp.xn--das-ist-grn-2hb.com
trace-server.prod-clustered.bugs.firebat.prime-video.amazon.dev
message-broker.at-735.veib.sab-test.comp.fahrziel-natur.de
netorgft4005979.mail.protection.outail.protection.sabzlearn.ir
message-broker.at-1952.veib.sab-test.comp.db-schenker.es
test.forgedemoservicedryrun.440926162108.f5eeaf5f.us-west-2.beta.forgeinternaldnsapps.ec2.dbschenker.pt
.
183.193.192.210.dynamic.dbcargo.de
122.218.132.202.dynamic.xn--dsseldorf-hauptbahnhof-slc.de
91.134.132.202.dynamic.omnitiq.de
message-broker.at-2685.veib.sab-test.comp.db-schenker.es
purd

In [44]:
# Add new regex patterns for the unique unmatched queries
for query in unmatched_queries:
  # Create a regex pattern for each unique unmatched query
  regex_patterns[f"unique_query_{query}"] = re.escape(query)

# Print the updated regex patterns
for name, pattern in regex_patterns.items():
  print(f"{name}: {pattern}")

service_specific_dns: ^_ldap\._tcp\.dc\._msdcs\.[A-Z]+\.[A-Z]+$
uuid_based_service_specific_dns: ^_ldap\._tcp\.[0-9a-fA-F-]{36}\.domains\._msdcs\.[A-Z]+\.[A-Z]+$
standard_hostname_domain: ^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$
complex_subdomain_structure: ^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$
standard_hostname_subdomain: ^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$
simple_domain: ^[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$
single_label_domain: ^[a-zA-Z0-9-]+$
extended_subdomain_structure: ^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$
another_extended_subdomain_structure: ^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$
ip_based_subdomain: ^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$
complex_ip_based_subdomain: ^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$
subd

In [45]:
import re
import pandas as pd


query_column = 'Query'

# Get the unique unmatched queries
unmatched_queries = df[query_column].unique()

# Initialize the regex patterns dictionary
regex_patterns = {}

# Function to automatically update regex patterns
def update_regex_patterns(unique_queries, regex_patterns):
  for query in unique_queries:
      # Create a regex pattern for each unique unmatched query
      regex_patterns[f"unique_query_{query}"] = re.escape(query)
  return regex_patterns

# Update the regex patterns with unique unmatched queries
regex_patterns = update_regex_patterns(unmatched_queries, regex_patterns)

# Print the updated regex patterns
for name, pattern in regex_patterns.items():
  print(f"{name}: {pattern}")

# Create a new column in the DataFrame to store the matched patterns
df['matched_pattern'] = None

# Iterate over the rows in the DataFrame
for index, row in df.iterrows():
  query = row[query_column]
  matched_pattern = None
  # Iterate over the updated regex patterns to find a match
  for name, pattern in regex_patterns.items():
    if re.search(pattern, query):
      matched_pattern = name
      break
  df.at[index, 'matched_pattern'] = matched_pattern

# Print the updated DataFrame with the matched patterns
print(df.head())

unique_query_fmcdg2.db-schenker.biz: fmcdg2\.db\-schenker\.biz
unique_query_ip-10-0-44-250.us-west-2.compute.regio-bayerisch-schwaben.de: ip\-10\-0\-44\-250\.us\-west\-2\.compute\.regio\-bayerisch\-schwaben\.de
unique_query_amazon.com: amazon\.com
unique_query_wikipedia.org: wikipedia\.org
unique_query_e.bank.dbdialog.eu: e\.bank\.dbdialog\.eu
unique_query_moncompte.cogeco.ca: moncompte\.cogeco\.ca
unique_query_www.sparrow-jnu.zlgl.eoffice.gov.in: www\.sparrow\-jnu\.zlgl\.eoffice\.gov\.in
unique_query_lunaful.yorouto.com: lunaful\.yorouto\.com
unique_query_sandbox-financing-v2.streckenkunde.net: sandbox\-financing\-v2\.streckenkunde\.net
unique_query_rgftomongodbqasubversion.blob.core.windows.net: rgftomongodbqasubversion\.blob\.core\.windows\.net
unique_query_tf-dev-pipeid-9b97a.datenbaeren-test.comp.transfracht.com: tf\-dev\-pipeid\-9b97a\.datenbaeren\-test\.comp\.transfracht\.com
unique_query_hq-rdgateway.dbbahnbau.de: hq\-rdgateway\.dbbahnbau\.de
unique_query_d19058ce9cc32a0c1edbc4

In [46]:
# Calculate the percentage of matched queries
num_matched_queries = df['matched_pattern'].notnull().sum()
total_queries = len(df)
percentage_matched_queries = (num_matched_queries / total_queries) * 100

# Print the percentage of matched queries
print(f"Percentage of matched queries: {percentage_matched_queries}%")

Percentage of matched queries: 100.0%


In [47]:
#Extracting the IP Address and saving it in txt file
# Function to extract IP addresses from the 'IP Address' column
def extract_ip_addresses(ip_string):
  # Use regex to find all valid IPv4 and IPv6 addresses
  ip_pattern = r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}|[0-9a-fA-F:]+)'
  return re.findall(ip_pattern, ip_string)

# Extract IP addresses from the 'IP Address' column
ip_addresses = selected_columns['IP Address'].apply(extract_ip_addresses)

# Flatten the list of lists and get unique IP addresses
unique_ips = set(ip for sublist in ip_addresses for ip in sublist)

# Save the unique IP addresses to a text file
with open('IP_list.txt', 'w') as file:
  for ip in unique_ips:
      file.write(f"{ip}\n")  # Write each IP on a new line

print("Unique IP addresses saved to IP_list.txt")

Unique IP addresses saved to IP_list.txt


In [48]:
#Extracting the domain and saving it in domain_list.txt
import re
import pandas as pd

# Function to extract domain from a query
def extract_domain(query):
  # Check if the query is already a simple domain
  if re.match(r'^[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$', query):
      return query  # Return the query as it is a valid domain

  # Use regex to match the domain part
  match = re.search(r'([a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,})$', query)
  if match:
      return match.group(1)  # Return the matched domain
  return None  # Return None if no match is found

# Apply the function to the 'Query' column
selected_columns['Domain'] = selected_columns['Query'].apply(extract_domain)

# Check for null values in the 'Domain' column
if selected_columns['Domain'].isnull().any():
    print("The 'Domain' column contains null values.")
else:
    print("The 'Domain' column does not contain null values.")

# Get the count of null values in the 'Domain' column
null_count = selected_columns['Domain'].isnull().sum()
print(f"The 'Domain' column contains {null_count} null values.")

# Save the domains to a text file
with open('domain_list.txt', 'w') as file:
  for domain in selected_columns['Domain'].unique():  # Get unique domains
      file.write(f"{domain}\n")  # Write each domain on a new line

print("Domains saved to domain_list.txt")

# Print the updated DataFrame with the extracted domains
print(selected_columns[['Query', 'Domain']])

The 'Domain' column contains null values.
The 'Domain' column contains 111 null values.
Domains saved to domain_list.txt
                                                  Query  \
0                                fmcdg2.db-schenker.biz   
1     ip-10-0-44-250.us-west-2.compute.regio-bayeris...   
2                                            amazon.com   
3                                         wikipedia.org   
4                                    e.bank.dbdialog.eu   
...                                                 ...   
3687                      p06-mailws.db-gebrauchtbus.de   
3688                                   api.facebook.com   
3689            www.fa-manuals.keinjobwiejederandere.de   
3690      ip-172-16-37-52.us-west-2.compute.db-cargo.lu   
3691                         nsgcloud.fahrziel-natur.de   

                                   Domain  
0                  fmcdg2.db-schenker.biz  
1     compute.regio-bayerisch-schwaben.de  
2                              amazon.

In [49]:
# Get the shape of the DataFrame
data_shape = selected_columns.shape

# Display the shape
print(f"Number of rows: {data_shape[0]}")
print(f"Number of columns: {data_shape[1]}")

Number of rows: 3692
Number of columns: 11


In [50]:
# Calculate the percentage of queries that were successfully extracted
num_extracted_queries = selected_columns['Domain'].notnull().sum()
total_queries = len(selected_columns)
percentage_extracted_queries = (num_extracted_queries / total_queries) * 100

print(f"Percentage of queries extracted: {percentage_extracted_queries:.2f}%")

Percentage of queries extracted: 96.99%


In [51]:
# Load the file without a header and specify custom column names
df = pd.read_csv('/content/domain_labels.txt',
               delimiter=',',
               header=None,
               names=['Domain', 'Categorized', 'Category', 'Risk'])

# Display the first few rows of the DataFrame
print(df.head())

                                Domain  Categorized Category Risk
0               fmcdg2.db-schenker.biz         True   Benign  Low
1  compute.regio-bayerisch-schwaben.de         True   Benign  Low
2                           amazon.com         True   Benign  Low
3                        wikipedia.org         True   Benign  Low
4                     bank.dbdialog.eu         True   Benign  Low


In [52]:
selected_columns.dtypes

,0
Timestamp,object
Client,object
Client ID,object
Query ID,float64
Query,object
Class,object
Record Type,object
Flags,object
IP Address,object
time,object


In [53]:
# Merge the extracted domains with the existing domain data
merged_df = pd.merge(selected_columns, df, on='Domain', how='inner', indicator=True)
merged_df = merged_df.drop(columns=['_merge'])

# Display the merged DataFrame

merged_df.head()


,Timestamp,Client,Client ID,Query ID,Query,Class,Record Type,Flags,IP Address,time,Domain,Categorized,Category,Risk
0,16-Apr-24,client,@0x7fccd4084398,-4.013320e+18,fmcdg2.db-schenker.biz,IN,A,+,(103.58.120.120),00:50:53.800,fmcdg2.db-schenker.biz,True,Benign,Low
1,16-Apr-24,client,@0x7fcd2c4568a8,-8.241670e+18,ip-10-0-44-250.us-west-2.compute.regio-bayeris...,IN,A,+,(103.58.120.120),00:50:53.800,compute.regio-bayerisch-schwaben.de,True,Benign,Low
2,16-Apr-24,client,@0x7fcd2c267ca8,-8.241670e+18,ip-10-0-51-77.us-east-2.compute.regio-bayerisc...,IN,A,+,(103.58.120.120),00:50:57.300,compute.regio-bayerisch-schwaben.de,True,Benign,Low
3,16-Apr-24,client,@0x7fcd2c267ca8,-8.241670e+18,ip-10-0-53-124.us-west-2.compute.regio-bayeris...,IN,A,+,(103.58.120.120),00:50:57.500,compute.regio-bayerisch-schwaben.de,True,Benign,Low
4,16-Apr-24,client,@0x7fcce41535b8,-8.235850e+18,amazon.com,IN,A,+,(103.58.120.120),00:50:53.900,amazon.com,True,Benign,Low


In [54]:
# Define mappings for Category and Risk
category_mapping = {
  'Malicious': 1,
  'Suspicious': 0.5,  # Optional if you want to include this category
  'Benign': 0
}

risk_mapping = {
  'High': 2,
  'Medium': 1,
  'Low': 0
}

# Convert Category and Risk to numerical values
merged_df['Category'] = merged_df['Category'].replace(category_mapping)
merged_df['Risk'] = merged_df['Risk'].replace(risk_mapping)

# Display the updated DataFrame
print(merged_df[['Domain', 'Category', 'Risk']])

                                   Domain  Category  Risk
0                  fmcdg2.db-schenker.biz         0     0
1     compute.regio-bayerisch-schwaben.de         0     0
2     compute.regio-bayerisch-schwaben.de         0     0
3     compute.regio-bayerisch-schwaben.de         0     0
4                              amazon.com         0     0
...                                   ...       ...   ...
3687                   acloud.yorouto.com         0     0
3688                  sxm.analyse-bahn.de         0     0
3689        p06-mailws.db-gebrauchtbus.de         0     0
3690  fa-manuals.keinjobwiejederandere.de         0     0
3691           nsgcloud.fahrziel-natur.de         0     0

[3692 rows x 3 columns]


In [55]:
import pandas as pd

# Clean up the 'IP Address' column by removing parentheses
# str.replace(r'[()]', '', regex=True) removes both opening and closing parentheses in one go
merged_df['IP Address'] = merged_df['IP Address'].str.replace(r'[()]', '', regex=True)

# Set the 'Timestamp' column as the index and ensure it's in datetime format
merged_df['Timestamp'] = pd.to_datetime(merged_df['Timestamp'])
merged_df.set_index('Timestamp', inplace=True)

# Get unique IP addresses in the DataFrame
unique_ip_addresses = merged_df['IP Address'].unique()

# Iterate through each unique IP address
for ip_address in unique_ip_addresses:
  # Filter the DataFrame by the current IP address
  filtered_df = merged_df[merged_df['IP Address'] == ip_address]

  # Ensure the DataFrame is sorted by index (timestamp) before resampling
  filtered_df = filtered_df.sort_index()

  # Resample the data to the desired time intervals and count the number of queries
  resampled_30min = filtered_df.resample('30T').size()
  resampled_1hour = filtered_df.resample('1H').size()
  resampled_1day = filtered_df.resample('1D').size()
  resampled_3days = filtered_df.resample('3D').size()

  # Combine the results into a DataFrame
  results = pd.DataFrame({
      '30min': resampled_30min,
      '1hour': resampled_1hour,
      '1day': resampled_1day,
      '3days': resampled_3days
  }).fillna(0).astype(int)

  # Print the results for the current IP address
  print(f"\nResampled results for IP address {ip_address}:")
  print(results)


Resampled results for IP address 103.58.120.120:
            30min  1hour  1day  3days
Timestamp                            
2024-04-16   3652   3652  3652   3652

Resampled results for IP address 127.0.0.1:
            30min  1hour  1day  3days
Timestamp                            
2024-04-16     31     31    31     31

Resampled results for IP address 2405:8a00:8001::20:
            30min  1hour  1day  3days
Timestamp                            
2024-04-16      9      9     9      9


In [56]:
merged_df.head()

,Client,Client ID,Query ID,Query,Class,Record Type,Flags,IP Address,time,Domain,Categorized,Category,Risk
Timestamp,,,,,,,,,,,,,
2024-04-16,client,@0x7fccd4084398,-4.013320e+18,fmcdg2.db-schenker.biz,IN,A,+,103.58.120.120,00:50:53.800,fmcdg2.db-schenker.biz,True,0,0
2024-04-16,client,@0x7fcd2c4568a8,-8.241670e+18,ip-10-0-44-250.us-west-2.compute.regio-bayeris...,IN,A,+,103.58.120.120,00:50:53.800,compute.regio-bayerisch-schwaben.de,True,0,0
2024-04-16,client,@0x7fcd2c267ca8,-8.241670e+18,ip-10-0-51-77.us-east-2.compute.regio-bayerisc...,IN,A,+,103.58.120.120,00:50:57.300,compute.regio-bayerisch-schwaben.de,True,0,0
2024-04-16,client,@0x7fcd2c267ca8,-8.241670e+18,ip-10-0-53-124.us-west-2.compute.regio-bayeris...,IN,A,+,103.58.120.120,00:50:57.500,compute.regio-bayerisch-schwaben.de,True,0,0
2024-04-16,client,@0x7fcce41535b8,-8.235850e+18,amazon.com,IN,A,+,103.58.120.120,00:50:53.900,amazon.com,True,0,0


In [57]:
# convert time to datetime format
merged_df['time'] = pd.to_datetime(merged_df['time'], format='%H:%M:%S.%f')

# convert datetime to desired format
merged_df['time'] = merged_df['time'].dt.strftime('%H:%M:%S.%f')

In [58]:
merged_df.dtypes

,0
Client,object
Client ID,object
Query ID,float64
Query,object
Class,object
Record Type,object
Flags,object
IP Address,object
time,object
Domain,object


In [59]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import ipaddress

# Assuming merged_df is your DataFrame
# Encode categorical variables into numerical variables
le_client_id = LabelEncoder()
le_query = LabelEncoder()
le_domain = LabelEncoder()
le_class = LabelEncoder()
le_record_type = LabelEncoder()
le_flags = LabelEncoder()

merged_df['Client ID'] = le_client_id.fit_transform(merged_df['Client ID'])
merged_df['Query'] = le_query.fit_transform(merged_df['Query'])
merged_df['Domain'] = le_domain.fit_transform(merged_df['Domain'])
merged_df['Class'] = le_class.fit_transform(merged_df['Class'])
merged_df['Record Type'] = le_record_type.fit_transform(merged_df['Record Type'])
merged_df['Flags'] = le_flags.fit_transform(merged_df['Flags'])
# Drop the 'Client' column
merged_df = merged_df.drop('Client', axis=1)

# Convert IP Address to numerical values
def ip_to_int(ip):
    try:
        return int(ipaddress.IPv4Address(ip))
    except ValueError:
        return int(ipaddress.IPv6Address(ip))

merged_df['IP Address'] = merged_df['IP Address'].apply(ip_to_int)

# Convert time to seconds since start of day
def time_to_seconds(time):
    h, m, s = time.split(':')
    if '.' in s:
        s, ms = s.split('.')
    else:
        ms = '0'
    return int(h) * 3600 + int(m) * 60 + int(s) + int(ms) / 1000

merged_df['time'] = merged_df['time'].apply(time_to_seconds)

# Verify there are no object types left (besides the label 'Categorized')
print(merged_df.dtypes)

# Scale the numerical data
scaler = StandardScaler()
merged_df[['Query ID', 'IP Address', 'time']] = scaler.fit_transform(merged_df[['Query ID', 'IP Address', 'time']])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(merged_df.drop('Categorized', axis=1), merged_df['Categorized'], test_size=0.2, random_state=42)

# Print the shapes of the training and testing sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

Client ID        int64
Query ID       float64
Query            int64
Class            int64
Record Type      int64
Flags            int64
IP Address      object
time           float64
Domain           int64
Categorized       bool
Category         int64
Risk             int64
dtype: object
Training set shape: (2953, 11) (2953,)
Testing set shape: (739, 11) (739,)


In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten,Input

# Define the model architecture
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))  # Use Input layer to specify the input shape
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [61]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3378 - loss: 129.7478 - val_accuracy: 1.0000 - val_loss: 8.6166e-06
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9918 - loss: 0.2272 - val_accuracy: 1.0000 - val_loss: 2.1900e-06
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9989 - loss: 0.0116 - val_accuracy: 1.0000 - val_loss: 1.8641e-06
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9996 - loss: 0.0039 - val_accuracy: 1.0000 - val_loss: 1.7982e-06
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9993 - loss: 0.0108 - val_accuracy: 1.0000 - val_loss: 1.7319e-06
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9985 - loss: 0.0134 - val_accuracy: 1.0000 - val_loss: 1.6605e-06
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9984 - loss: 0.0049 - val_accuracy: 1.0000 - val_loss: 1.6005e-06
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9988 - loss: 0.0448 - v

In [62]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.3f}')
print(f'Test Accuracy: {accuracy:.3f}')

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 8.1279e-07 
Test Loss: 0.000
Test Accuracy: 1.000


In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

        True       1.00      1.00      1.00       739

    accuracy                           1.00       739
   macro avg       1.00      1.00      1.00       739
weighted avg       1.00      1.00      1.00       739



In [67]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D, Input

# Reshape data to add an extra dimension for Conv1D
X = merged_df.drop('Categorized', axis=1).values
X = X.reshape((X.shape[0], X.shape[1], 1))  # Adding an extra dimension
y = merged_df['Categorized'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the training and testing sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

# Define the CNN model architecture
model = Sequential()
model.add(Input(shape=(X_train.shape[1], 1)))  # Use Input layer to specify the input shape
model.add(Conv1D(64, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary outputs (0 or 1)

# Calculate and print metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

Training set shape: (2953, 11, 1) (2953,)
Testing set shape: (739, 11, 1) (739,)
Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9581 - loss: 1.6386 - val_accuracy: 1.0000 - val_loss: 4.6909e-07
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 5.8272e-05 - val_accuracy: 1.0000 - val_loss: 2.2675e-07
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 1.4183e-05 - val_accuracy: 1.0000 - val_loss: 2.0283e-07
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 3.2582e-05 - val_accuracy: 1.0000 - val_loss: 1.9289e-07
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 4.0181e-05 - val_accuracy: 1.0000 - val_loss: 1.3750e-07
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.2686e-05 - val_accuracy: 1.0000 - val_loss: 1.0507e-07
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 1.4585e-04 - val_accuracy: 1.0000 - val_l

In [68]:
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input
# Reshape data to add an extra dimension for LSTM
X = merged_df.drop('Categorized', axis=1).values
X = X.reshape((X.shape[0], X.shape[1], 1))  # Adding an extra dimension
y = merged_df['Categorized'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the training and testing sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

# Define the LSTM model architecture
model = Sequential()
model.add(Input(shape=(X_train.shape[1], 1)))  # Input shape for LSTM
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary outputs (0 or 1)

# Calculate and print metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

Training set shape: (2953, 11, 1) (2953,)
Testing set shape: (739, 11, 1) (739,)
Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 46ms/step - accuracy: 0.9535 - loss: 0.3656 - val_accuracy: 1.0000 - val_loss: 0.0266
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 1.0000 - loss: 0.0202 - val_accuracy: 1.0000 - val_loss: 0.0056
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 1.0000 - loss: 0.0055 - val_accuracy: 1.0000 - val_loss: 0.0029
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 1.0000 - loss: 0.0033 - val_accuracy: 1.0000 - val_loss: 0.0020
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 1.0000 - loss: 0.0024 - val_accuracy: 1.0000 - val_loss: 0.0015
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 1.0000 - loss: 0.0019 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 8/10
24/24 ━━━━━━━━━━━━